In [1]:
import os
from glob import glob
import threading

import numpy as np
import dask.dataframe as dd
from dask import delayed
import pandas as pd
import xarray as xr

%matplotlib inline
from matplotlib import pyplot as plt
#import matplotlib.animation as anim

import crocosi.postp as pp
from crocosi.jet import set_relevant_time

/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.7/site-packages/distributed/utils.py:133: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  RuntimeWarning,


In [ ]:
from dask_jobqueue import PBSCluster
cluster = PBSCluster()
w = cluster.scale(28*2)

In [2]:
from dask.distributed import Client
client = Client()  # set up local cluster on your laptop
client

Client Scheduler: tcp://127.0.0.1:38748 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 56 Memory: 107.37 GB


In [3]:
root_path = '/home/datawork-lops-osi/slgentil/croco/jetn/'
#run = 'jet_cfg1_wp9_4km_0a2000j/'
run = 'jet_cfg1_wp75_4km_1500a2000j_floats_lev50_itide/'
#file='t1/file_his_00010101-00010125.nc'
#File='t1'
gparams = {'f0': 1.0313e-4, 'beta': 1.6186e-11}
r = pp.CROCOrun(root_path+run, prefix='file_', open_nc=['surf'], grid_params=gparams)

In [4]:
df = dd.read_csv(glob(root_path+run+'t1/float.????'), 
                 names=['id','time','xgrid','ygrid','zgrid','depth','temp','u','v','dudt','dvdt','pres'],
                 delim_whitespace=True)#.persist()
dxy = int(run[run.find('km')-1])
df['x'] = df.xgrid*dxy
df['y'] = df.ygrid*dxy
df = df.persist()
df.head()

,id,time,xgrid,ygrid,zgrid,depth,temp,u,v,dudt,dvdt,pres,x,y
0,7161,1500.0069,196.40944,541.79652,50.0,-0.935944,25.886666,0.058079,-0.105506,-5.820925e-08,3.420817e-07,-8.260263,785.63776,2167.18608
1,7162,1500.0069,196.40937,543.30184,50.0,-0.940022,25.885229,0.057283,-0.101750,-2.034746e-08,3.515808e-07,-8.300321,785.63748,2173.20736
2,7163,1500.0069,196.40946,544.80716,50.0,-0.944117,25.884374,0.058512,-0.097537,-1.201106e-09,3.599363e-07,-8.340519,785.63784,2179.22864
3,7164,1500.0069,196.40961,546.31250,50.0,-0.948345,25.884627,0.060480,-0.093268,-1.958836e-08,3.627984e-07,-8.381996,785.63844,2185.25000
4,7165,1500.0069,196.40971,547.81780,50.0,-0.952706,25.885603,0.061626,-0.089432,-6.096121e-08,3.575484e-07,-8.424740,785.63884,2191.27120


distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


### Paramètres :

In [5]:
vari = 'ssh'
vari1 = 'u'
vari2 = 'v'
floats_var = 'depth'
#float_id = [4000,300]

In [6]:
#flt = df.groupby('id').get_group(float_id[0]).compute().sort_values('time').set_index('time')
flt1 = df.groupby('id').get_group(225).compute().sort_values('time').set_index('time')

In [ ]:
#Décale les données du fichier surf sur les mêmes temps que les données des floats ( décalage de 5min sinon):
#v1 = (r['surf'][vari1].shift(time=-1)+r['surf'][vari1])*.5
#v2 = (r['surf'][vari2].shift(time=-1)+r['surf'][vari2])*.5
#v0 = (r['surf'][vari].shift(time=-1)+r['surf'][vari])*.5

###Génération des figures de ti (temps initial en jours) à tf (temps final en jours)
tous les Dt (intervalle de temps voulu en heures *6 => un pas de tps=10min)

In [26]:
figdir = 'home1/datahome/zcaspar/figs/'
print('Figures directory: '+figdir)

def genfig(i, ds, v, flt, n, size, overwrite=True, offline=True):
    
    figname = figdir+'float_%s_t%05d_%s' %(v, n, size)+'.png'
    
    if not os.path.isfile(figname) or overwrite:
        time = ds['time_instant'].values
        
        #
        MPL_LOCK = threading.Lock()
        with MPL_LOCK:
            plt.switch_backend('agg')    
            #
            flti = flt.loc[i]
            fig = plt.figure(figsize=(10,10))
            grid = plt.GridSpec(3, 1, wspace=0.4, hspace=0.3)
    
            ax = plt.subplot(grid[2, :])
    #off1 = .5
            ax.plot(flt.index,flt.u)
            ax.axvline(x=i, color='k', lw=1)
            ax.set_xlabel('Time (days)');ax.set_ylabel(r'Zonal velocity ($m.s^{-1}$)')
            ax = plt.subplot(grid[:2, :])    
            ax.plot(flt.x, flt.y)
   
            ax.plot(flti.x, flti.y, marker='d', ms=8, 
                    markeredgecolor='k')
            ds.plot.contourf(ax=ax, levels = np.linspace(-1,0,20), cmap='RdBu_r')
            ax.set_xlabel(r'$x_\rho$');ax.set_ylabel(r'$y_\rho$')
            ax.set_title(r'Time$\simeq$%sdays'%np.around(i,decimals=1))
            if offline:
                fig.savefig(figname)
                plt.close()
            #
            m = 1.
    else:
        m = -1.
    return m

Figures directory: home1/datahome/zcaspar/figs/


In [8]:
ds = r['surf']['ssh'].to_dataset()

In [9]:
ds = ds.assign_coords(time=ds.time_counter, x_rho = ds.x_rho*dxy,y_rho = ds.y_rho*dxy)

In [10]:
ds

<xarray.Dataset>
Dimensions:       (time: 72000, x_rho: 258, y_rho: 722)
Coordinates:
    eta_rho       (y_rho, x_rho) float32 dask.array<chunksize=(722, 258), meta=np.ndarray>
    xi_rho        (y_rho, x_rho) float32 dask.array<chunksize=(722, 258), meta=np.ndarray>
    time_instant  (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    time_counter  (time) float64 1.5e+03 1.5e+03 1.5e+03 ... 2e+03 2e+03 2e+03
    f             (y_rho, x_rho) float32 dask.array<chunksize=(722, 258), meta=np.ndarray>
  * time          (time) float64 1.5e+03 1.5e+03 1.5e+03 ... 2e+03 2e+03 2e+03
  * x_rho         (x_rho) int64 0 4 8 12 16 20 ... 1008 1012 1016 1020 1024 1028
  * y_rho         (y_rho) int64 0 4 8 12 16 20 ... 2864 2868 2872 2876 2880 2884
Data variables:
    ssh           (time, y_rho, x_rho) float32 dask.array<chunksize=(1, 722, 258), meta=np.ndarray>

In [11]:
flt_id = 225
time0 = 1550
time1 = 1551

In [22]:
ldf = df[ (df.time>=time0) & (df.time<=time1) ]
ldf = ldf[ (ldf.id==flt_id) ]
xmin,xmax,ymin,ymax = [ldf.xgrid.min().compute(),ldf.xgrid.max().compute()
                       ,ldf.ygrid.min().compute(),ldf.ygrid.max().compute()]
dsi = ds.sel(x_rho=slice(xmin,xmax),y_rho=slice(ymin,ymax),time=slice(time0,time1))


In [25]:
Xgrid = ldf.xgrid.compute()
Ygrid = ldf.ygrid.compute()
flt_u = ldf.X1.compute()

In [30]:
Time = ldf.time.compute()

In [12]:
rg = np.arange(6*24*50,6*24*51) # 3 days
II = np.array_split(rg,len(rg)/56) # 4 maybe will work
print('%d loops to be done'%len(II))

2 loops to be done


In [24]:
dsi = ds[vari].sel(x_rho=slice(int(flt1.x.min()),
                                int(flt1.x.max()))
                   ,y_rho=slice(int(flt1.y.min())
                                ,int(flt1.y.max()))
                  ,time=slice(1500,1600))

In [25]:
dsi

<xarray.DataArray 'ssh' (time: 14400, y_rho: 85, x_rho: 256)>
dask.array<getitem, shape=(14400, 85, 256), dtype=float32, chunksize=(1, 85, 256), chunktype=numpy.ndarray>
Coordinates:
    eta_rho       (y_rho, x_rho) float32 dask.array<chunksize=(85, 256), meta=np.ndarray>
    xi_rho        (y_rho, x_rho) float32 dask.array<chunksize=(85, 256), meta=np.ndarray>
    time_instant  (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    time_counter  (time) float64 1.5e+03 1.5e+03 1.5e+03 ... 1.6e+03 1.6e+03
    f             (y_rho, x_rho) float32 dask.array<chunksize=(85, 256), meta=np.ndarray>
  * time          (time) float64 1.5e+03 1.5e+03 1.5e+03 ... 1.6e+03 1.6e+03
  * x_rho         (x_rho) int64 4 8 12 16 20 24 ... 1008 1012 1016 1020 1024
  * y_rho         (y_rho) int64 1704 1708 1712 1716 1720 ... 2028 2032 2036 2040
Attributes:
    long_name:           free-surface
    units:               meter
    online_operation:    instant
    interval_operation:  300 s
    interval_

In [28]:
n =0
for I in II:
    print('  %d - %d'%(I[0],I[-1]))
    #print(ds.time.isel(time=i))
    #lt = ds.time_instant.isel(time=I).values
    #ldf = df[ (df.time>lt[0]-2) & (df.time<lt[-1]) ]
    values = [genfig(i, dsi.sel(time=i,method='nearest'), 'ssh', \
                              flt1.loc[time0:time1], n,\
                              'zoom', overwrite=True, offline=False) for i in flt1.index[I]]
    values1 = [genfig(i, ds.sel(time=i,method='nearest'), 'ssh', \
                              flt1.loc[time0:time1], n, \
                              'large', overwrite=True, offline=False) for i in flt1.index[I]]
    n+=len(I)
    #futures = client.compute(values)
    #results = client.gather(futures)

  7200 - 7271


distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)


KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [ ]:
n =0
for I in II:
    print('  %d - %d'%(I[0],I[-1]))
    #print(ds.time.isel(time=i))
    #lt = ds.time_instant.isel(time=I).values
    #ldf = df[ (df.time>lt[0]-2) & (df.time<lt[-1]) ]
    values = [genfig(i, dsi.sel(time=i,method='nearest'), 'ssh', \
                              flt1.loc[time0:time1], \
                              overwrite=True, offline=True) for i in flt1.index[I]]
    n+=len(I)
    #futures = client.compute(values)
    #results = client.gather(futures)

In [34]:
cluster.close()